In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import missingno as msno

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import RandomForestClassifier

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import LeaveOneGroupOut, GridSearchCV

from sklearn.inspection import permutation_importance

from sklearn.metrics import make_scorer, roc_auc_score
import os
%matplotlib inline

In [ ]:
def score_func(y, y_pred):
    score = roc_auc_score(y, y_pred, multi_class="ovr")
    return score

my_scores = make_scorer(score_func=score_func, greater_is_better=True, needs_proba=True, needs_threshold=False)

In [ ]:
os.getcwd()

In [ ]:
os.chdir("../../3_Result/1_Logistic/2_Trials-Back")

In [ ]:
os.getcwd()

In [ ]:
df_noref = pd.read_csv("/Users/zhengyuanrui/Decoding_SALT/Decode_SALT/2_Data/df_no_ref.csv")
df_selfref = pd.read_csv("/Users/zhengyuanrui/Decoding_SALT/Decode_SALT/2_Data/df_self_ref.csv")

In [ ]:
df_noref.head()

In [ ]:
df_selfref.head()

In [ ]:
msno.matrix(df_noref, labels=True)

In [ ]:
msno.matrix(df_selfref, labels=True)

In [ ]:
X_norefb = df_noref.iloc[:, 11:-1].values
X_selfrefb = df_selfref.iloc[:, 12:-1].values


y_noref = df_noref["label"].values
y_selfref = df_selfref["label"].values

norefcolb = df_noref.iloc[:, 11:-1].columns
selfrefcolb = df_selfref.iloc[:, 12:-1].columns

In [ ]:
df_selfref.iloc[:, 12:-1]

In [ ]:
df_noref.iloc[:, 8:11]

In [ ]:
X_norefb.shape

In [ ]:
X_selfrefb.shape

In [ ]:
y_noref.shape

In [ ]:
y_selfref.shape

In [ ]:
groups_no = df_noref["Subject"].values
groups_self = df_selfref["Subject"].values

In [ ]:
logo = LeaveOneGroupOut()

In [ ]:
def lr_within_task(X, y, group, source):
    df_result = dict(subID=[], score=[], source=[], target=[])# source拟合的，target预测的condition
    feature_importance = []
    feature_coef = []
    for train, test in logo.split(X, y, groups=group):
        test_sub = np.unique(group[test])[0]
        df_result["subID"].append(test_sub)
        
        logi = make_pipeline(
            MinMaxScaler(), 
            LogisticRegressionCV(Cs = np.logspace(-6, 3, 7), cv = 5, class_weight='balanced', 
                                 random_state=123, max_iter=5000, multi_class="ovr"))
        
        logi.fit(X=X[train], y=y[train])
        feature_coef.append(logi.steps[-1][-1].coef_)
        im = permutation_importance(logi, X[test], y[test], scoring=my_scores, n_repeats=20, n_jobs=-1, random_state=123)
        feature_importance.append(im['importances_mean'])
        y_pred = logi.predict_proba(X[test])
        score = roc_auc_score(y[test], y_pred, multi_class='ovr')

        df_result['score'].append(score)
        df_result['source'].append(source)
        df_result['target'].append(source)

    return pd.DataFrame(df_result), feature_importance, feature_coef


In [ ]:
def lr_cross_task(X_source, y_source, X_target, y_target, target_group, source_name, target_name):
    df_result = dict(subID=[], score=[], source=[], target=[])# source拟合的，target预测的condition
    feature_importance = []
    feature_coef = []
    logi = make_pipeline(
            MinMaxScaler(), 
            LogisticRegressionCV(Cs = np.logspace(-6, 3, 7), cv = 5, class_weight='balanced', 
                                 random_state=123, max_iter=5000, multi_class="ovr"))

    logi.fit(X=X_source, y=y_source)


    for sub in np.unique(target_group):
        idx_sub = target_group == sub
        feature_sub = X_target[idx_sub]
        label_sub = y_target[idx_sub]

        im = permutation_importance(logi, feature_sub, label_sub, scoring=my_scores, n_repeats=20, n_jobs=-1, random_state=123)
        feature_importance.append(im['importances_mean'])
        feature_coef.append(logi.steps[-1][-1].coef_)
        y_pred = logi.predict_proba(feature_sub)
        score = roc_auc_score(label_sub, y_pred, multi_class="ovr")

        df_result['subID'].append(sub)
        df_result["score"].append(score)
        df_result["source"].append(source_name)
        df_result["target"].append(target_name)


    return pd.DataFrame(df_result), feature_importance, feature_coef

In [ ]:
score_no, im_no, coef_no = lr_within_task(X = X_norefb, y = y_noref, group = groups_no, source="No_Ref")
score_self, im_self, coef_self = lr_within_task(X = X_selfrefb, y = y_selfref, group = groups_self, source="Self_Ref")

In [ ]:
df_im_no = pd.DataFrame(np.array(im_no), columns=norefcolb)
df_im_no

In [ ]:
#No ref to self ref trial back
df_cross1, im_cross1, coef_cross1 = lr_cross_task(X_source=X_norefb, y_source=y_noref, X_target=X_selfrefb, y_target=y_selfref, target_group=groups_self, source_name="No_Ref", target_name="Self_Ref")
#self to no ref trial back
df_cross2, im_cross2, coef_cross2 = lr_cross_task(X_source=X_selfrefb, y_source=y_selfref, X_target=X_norefb, y_target=y_noref, target_group=groups_no, source_name="Self_Ref", target_name="No_Ref")

In [ ]:
df_score = pd.concat([score_no, score_self])

In [ ]:
df_score.to_csv("LR_withinscore.csv")

In [ ]:
df_cross_score = pd.concat([df_cross1, df_cross2])

In [ ]:
df_cross_score.to_csv("LR_crossscore.csv")

In [ ]:
df_im_no = pd.DataFrame(im_no, columns=norefcolb)
df_im_no["subj_idx"] = np.arange(1, len(im_no)+1)
df_im_no["source"] = "No_Ref"
df_im_no["target"] = "No_Ref"

df_im_self = pd.DataFrame(im_self, columns=selfrefcolb)
df_im_self["subj_idx"] = np.arange(1, len(im_self)+1)
df_im_self["source"] = "Self_Ref"
df_im_self["target"] = "Self_Ref"

In [ ]:
im_no_long = pd.melt(df_im_no, id_vars=["subj_idx", "source", "target"])
im_self_long = pd.melt(df_im_self, id_vars=["subj_idx", "source", "target"])
im_within = pd.concat([im_no_long, im_self_long])
im_within

In [ ]:
im_within.to_csv("importance_within.csv")

In [ ]:
df_im_cross1 = pd.DataFrame(im_cross1, columns=selfrefcolb)
df_im_cross1["subj_idx"] = np.arange(1, len(im_cross1)+1)
df_im_cross1["source"] = "No_Ref"
df_im_cross1["target"] = "Self_Ref"

df_im_cross2 = pd.DataFrame(im_cross2, columns=norefcolb)
df_im_cross2["subj_idx"] = np.arange(1, len(im_cross2)+1)
df_im_cross2["source"] = "Self_Ref"
df_im_cross2["target"] = "No_Ref"

In [ ]:
im_cross1_long = pd.melt(df_im_cross1, id_vars=["subj_idx", "source", "target"])
im_cross2_long = pd.melt(df_im_cross2, id_vars=["subj_idx", "source", "target"])
im_cross = pd.concat([im_cross1_long, im_cross2_long])
im_cross

In [ ]:
im_cross.to_csv("importance_cross.csv")

In [ ]:
np.save('coef_no.npy', coef_no)
np.save('coef_self.npy', coef_self)
np.save('coef_cross1.npy', coef_cross1)
np.save('coef_cross2.npy', coef_cross2)